# prepare images

In [1]:
#import packages
from PIL import Image #Bildbearbeitungspacket (PIL = Pillow)
import numpy as np
import os # provides functions for creating and removing a directory (folder), fetching its contents, changing and identifying the current directory, etc
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf 
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model

In [2]:
def read_traindata(path):
    images = []
    labels = []
    for image in sorted(os.listdir(str(path))):
        im = Image.open(str(path)+str(image))
        im = im.resize((200,200)) #resize to get all images to the same dimension
        images.append(np.array(im))
        if image[4] == 'M':
            labels.append(True)
        else:
            labels.append(False)
                
    return images, labels


In [3]:
#read in trainings and test images

train_images, train_labels = read_traindata('fold1/train/400X/')

test_images, test_labels = read_traindata('fold1/test/400X/')


In [4]:
#convert train_images, train_labels, test_images and test_labels to a numpy array
train_images = np.array(train_images) 
train_labels = np.array(train_labels) 
test_images = np.array(test_images)

#reshape train_labels and test_lab
train_labels = np.reshape(train_labels,(-1,1)) #-1 for unknown dimension

In [5]:
#conver train_labels to one-hot encoding
#hierbei wird aus [False] [1. 0.] und aus [True] [0. 1.]

enc = OneHotEncoder() #a new binary variable is added for each unique integer value
train_lab = enc.fit_transform(train_labels).toarray()


In [6]:
# normalize images by dividing them with the maximum number of pixel
# jetzt liegen die Werte zwischen 0 und 1

train_im = train_images / 255.0
test_im = test_images / 255.0

In [7]:
#build model
buffer_size = len(train_im) 
print(buffer_size)
train_ds = tf.data.Dataset.from_tensor_slices((train_im, train_lab)).shuffle(buffer_size).batch(32) #rain_ds has two tensors that are randomly sampled and batched. These tensors represent the training images and labels.


1165
<BatchDataset element_spec=(TensorSpec(shape=(None, 200, 200, 3), dtype=tf.float64, name=None), TensorSpec(shape=(None, 2), dtype=tf.float64, name=None))>


# CNN Model

In [8]:
#define class CNN_Model by using Model and modifying it
# CNN_Model can identify patterns and is therefore usefull for Image analysis
# we use a convolutional/faltendes Neural Network (CNN) with dense layers for class prediction
# A convolutional layer has a number of filters that do convolutional operations. 


class CNN_Model(Model):
    def __init__(self):
        super(CNN_Model, self).__init__()
        self.conv1 = Conv2D(32, 3, padding='same', activation='relu') #2D convolution layer (e.g. spatial convolution over images). --> Filter der über das Bild bewegt wird
        self.pool1 = MaxPool2D((2,2)) #Max pooling operation for 2D spatial data. --> nimmt größten Wert im Fenster 2x2 --> viertelt Anzahl an Werten
        self.conv2 = Conv2D(64, 3, padding='same', activation='relu')
        self.pool2 = MaxPool2D((2,2))
        self.flatten = Flatten() #Flattens the input. Does not affect the batch size. --> "bild wird langgezogen --> 1 dimensional"
        self.d1 = Dense(512, activation='relu') #normal NN layer, with relu as an activation function
        self.dropout1 = Dropout(0.4) # Applies Dropout to the input.
        self.d2 = Dense(128, activation='relu') 
        self.dropout2 = Dropout(0.4)
        self.d3 = Dense(43, activation='relu')
        self.d4 = Dense(2, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.dropout1(x)
        x = self.d2(x)
        x = self.dropout2(x)
        x = self.d3(x)
        x = self.d4(x)
        return x
    
model_CNN = CNN_Model()

In [9]:
#create an object for our model
#we use  Adam as our optimizer
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

#define our loss functions
#We use categorical cross-entropy as our loss function 
train_loss = tf.keras.metrics.Mean(name='train_loss') # mean value of all losses for each epoch
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy') #the accuracy measure of our model for each epoch

In [10]:
@tf.function #this decorator converts the function into a graph.
def train_step(images, labels):
    'function to train our model and computes the loss and gradients'
    with tf.GradientTape() as tape:
        'tf.GradientTape() is a high-level API that is used to compute differentiations'
        predictions = model_CNN(images)
        loss = loss_object(labels, predictions)
        
        gradients = tape.gradient(loss, model_CNN.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model_CNN.trainable_variables))
        
        train_loss(loss)
        train_accuracy(labels, predictions)

In [11]:
#build and train our model
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
        
    model_CNN.save_weights('./content', save_format='tf')
    
    print('Epoch:',str(epoch+1),' Loss:',str(train_loss.result()),' Accuracy:', str(train_accuracy.result()*100))
    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch: 1  Loss: tf.Tensor(1.1916065, shape=(), dtype=float32)  Accuracy: tf.Tensor(66.09442, shape=(), dtype=float32)
Epoch: 2  Loss: tf.Tensor(0.5641064, shape=(), dtype=float32)  Accuracy: tf.Tensor(72.36052, shape=(), dtype=float32)
Epoch: 3  Loss: tf.Tensor(0.5259005, shape=(), dtype=float32)  Accuracy: tf.Tensor(71.75966, shape=(), dtype=float32)
Epoch: 4  Loss: tf.Tensor(0.5248934, shape=(), dtype=float32)  Accuracy: tf.Tensor(76.30901, shape=(), dtype=float32)
Epoch: 5  Loss: tf.Tensor(0.51930445, shape=(), dtype=float32)  Accuracy: tf.Tensor(72.96137, shape=(), dtype=float32)


In [12]:
predictions = np.argmax(model_CNN(test_im),axis=1)

In [13]:
correct = predictions == test_labels
print(sum(correct)/len(correct))

0.816793893129771


# fully connected NN Model


In [37]:
#fully connected --> erst flatten, dann dense
# we use a fully connected neural network
#https://medium.com/swlh/fully-connected-vs-convolutional-neural-networks-813ca7bc6ee5
#fully connected:
# connect every neuron in one layer to every neuron in the other layer.

class fully_c_Model(Model):
    def __init__(self):
        super(fully_c_Model, self).__init__()
        self.flatten = Flatten()
        self.d0 = Dense(2330, activation='relu') #2048 = number of weights
        self.d1 = Dense(1165, activation='relu')
        self.d2 = Dense(582, activation='relu')
        self.d3 = Dense(291, activation='relu')
        self.d4 = Dense(146, activation='relu')
        self.d5 = Dense(2, activation='softmax')
        
    def call(self, x):
        x = self.flatten(x) 
        x = self.d0(x)
        x = self.d1(x)
        x = self.d2(x)
        x = self.d3(x)
        x = self.d4(x)
        x = self.d5(x)
        return x
    
model_fully_c = fully_c_Model()

In [38]:
optimizer = tf.keras.optimizers.Adam()

In [39]:
@tf.function #this decorator converts the function into a graph.
def train_step(images, labels):
    'function to train our model and computes the loss and gradients'
    with tf.GradientTape() as tape:
        'tf.GradientTape() is a high-level API that is used to compute differentiations'
        predictions = model_fully_c(images)
        loss = loss_object(labels, predictions)
        
        gradients = tape.gradient(loss, model_fully_c.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model_fully_c.trainable_variables))
        
        train_loss(loss)
        train_accuracy(labels, predictions)

In [40]:
#build and train our model
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
        
    model_fully_c.save_weights('./content', save_format='tf')
    
    print('Epoch:',str(epoch+1),' Loss:',str(train_loss.result()),' Accuracy:', str(train_accuracy.result()*100))
    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch: 1  Loss: tf.Tensor(29.817862, shape=(), dtype=float32)  Accuracy: tf.Tensor(53.99142, shape=(), dtype=float32)
Epoch: 2  Loss: tf.Tensor(2.5358858, shape=(), dtype=float32)  Accuracy: tf.Tensor(61.37339, shape=(), dtype=float32)
Epoch: 3  Loss: tf.Tensor(0.6806833, shape=(), dtype=float32)  Accuracy: tf.Tensor(65.92275, shape=(), dtype=float32)
Epoch: 4  Loss: tf.Tensor(0.5686251, shape=(), dtype=float32)  Accuracy: tf.Tensor(75.19313, shape=(), dtype=float32)
Epoch: 5  Loss: tf.Tensor(0.50332826, shape=(), dtype=float32)  Accuracy: tf.Tensor(78.45493, shape=(), dtype=float32)


In [41]:
predictions = np.argmax(model_fully_c(test_im),axis=1)

In [42]:
correct = predictions == test_labels
print(sum(correct)/len(correct))

0.6213740458015267


# shallow NN model 3

In [27]:
from tensorflow.keras.layers import Embedding, LSTM
lstm = tf.keras.layers.LSTM(4)

In [29]:
class shallow_Model(Model):
    def __init__(self):
        super(shallow_Model, self).__init__()
        self.flatten = Flatten()
        self.d1 = Dense(2000, activation='relu')
        self.d2 = Dense(2, activation='softmax')
       
        
    def call(self, x): 
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d2(x)
        return x
    
model_shallow = shallow_Model()

In [30]:
optimizer = tf.keras.optimizers.Adam()

In [32]:
@tf.function #this decorator converts the function into a graph.
def train_step(images, labels):
    'function to train our model and computes the loss and gradients'
    with tf.GradientTape() as tape:
        'tf.GradientTape() is a high-level API that is used to compute differentiations'
        predictions = model_shallow(images)
        loss = loss_object(labels, predictions)
        
        gradients = tape.gradient(loss, model_shallow.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model_shallow.trainable_variables))
        
        train_loss(loss)
        train_accuracy(labels, predictions)

In [34]:
#build and train our model
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
        
    model_shallow.save_weights('./content', save_format='tf')
    
    print('Epoch:',str(epoch+1),' Loss:',str(train_loss.result()),' Accuracy:', str(train_accuracy.result()*100))
    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch: 1  Loss: tf.Tensor(45.85336, shape=(), dtype=float32)  Accuracy: tf.Tensor(64.67811, shape=(), dtype=float32)
Epoch: 2  Loss: tf.Tensor(5.3578353, shape=(), dtype=float32)  Accuracy: tf.Tensor(68.8412, shape=(), dtype=float32)
Epoch: 3  Loss: tf.Tensor(2.4574704, shape=(), dtype=float32)  Accuracy: tf.Tensor(69.87125, shape=(), dtype=float32)
Epoch: 4  Loss: tf.Tensor(2.081162, shape=(), dtype=float32)  Accuracy: tf.Tensor(68.669525, shape=(), dtype=float32)
Epoch: 5  Loss: tf.Tensor(2.101782, shape=(), dtype=float32)  Accuracy: tf.Tensor(68.06867, shape=(), dtype=float32)


In [35]:
predictions = np.argmax(model_shallow(test_im),axis=1)

In [36]:
correct = predictions == test_labels
print(sum(correct)/len(correct))

0.5572519083969466
